In [4]:
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
!pip install -U -r requirements.txt

import tkinter as tk
import cv2
from PIL import Image, ImageTk
import torch
from torchvision.transforms import functional as F
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox

class ObjectDetector:
    def __init__(self, model_path='best.pt'):
        self.model = attempt_load(model_path, map_location=torch.device('cpu')).fuse().eval()
        self.img_size = 640  # set default image size
        self.conf_thres = 0.4  # set default confidence threshold
        self.iou_thres = 0.5  # set default IoU threshold

    def detect_objects(self, frame):
        img = letterbox(frame, new_shape=self.img_size)[0]
        img = F.normalize(F.to_tensor(img), 0.5, 0.5)
        img = img.unsqueeze(0)

        pred = self.model(img)[0]
        pred = non_max_suppression(pred, self.conf_thres, self.iou_thres)[0]

        if pred is not None:
            pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], frame.shape).round()
        return pred

class App:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.video_source = 0
        self.detector = ObjectDetector()

        self.canvas = tk.Canvas(window, width=640, height=480)
        self.canvas.pack()

        self.btn_quit = tk.Button(window, text="Quit", width=10, command=self.on_quit)
        self.btn_quit.pack(anchor=tk.CENTER, expand=True)

        self.delay = 15
        self.update()

        self.window.mainloop()

    def update(self):
        ret, frame = self.get_frame()
        if ret:
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        self.window.after(self.delay, self.update)

    def get_frame(self):
        cap = cv2.VideoCapture(self.video_source)
        if not cap.isOpened():
            return False, None
        ret, frame = cap.read()
        if ret:
            pred = self.detector.detect_objects(frame)
            # Draw bounding boxes on the frame
            if pred is not None:
                for *xyxy, conf, cls in pred:
                    cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
            return ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            return ret, None

    def on_quit(self):
        self.window.destroy()

App(tk.Tk(), "YOLOv5 Object Detection")



SyntaxError: invalid syntax (159329990.py, line 1)